NLP 2015 | 풀잎스쿨

## Abstract

1. Text classification에서 Character-level convolutional networks(ConvNets)를 BoW, n-gram, TFIDF variants, word-based ConvNets, RNN과 비교한다.
2. 여러 데이터 셋에서 SOTA?
3. 기존방식이랑 비교해 보았다.

## Introduction

Text classification는 free-text documents에 predefined category를 부여하는 NLP영역의 토픽입니다. Text Classification연구는 ML classifier로 넣어 줄 최적의 feature를 정하는 것에서 시작되었다. 
<br>
본 논문에서는, text를 raw signal로 받기 위해 character level model을 사용했고, 1-D convolution 을 사용하다. 여태까지 CNN 모델들은 input의 단위는 단어였다(embedded word vector)였다. 하지만 이번 논문에서는 character 단위의 CNN을 사용해서 문서 분류 문제를 해결하고자 했다. 
<br>
데이터 셋이 매우 큰 경우, ConvNet은 단어에 대한 정보(통사론 단위의 의미론적인 정보를 포함한다.)를 필요하지 않는다. 이렇게 character 단위로 만들어진 모델은 조금 수정으로도 여러 언어에 적용 가능하고, 오타나 이모티콘 같은 일반적인 단어와 마찬가지로 잘 학습이 된다.

## Character-Level Convolution Networks

__Key-Module__

가장 핵심적인 부분은 temporal convlutional 모듈이다. 이것은 1-D 컨볼류션으로, discrete 한 input과 kernel 을 가지게 된다. 그러면 Convlution을 다음 과 같은 함수로 표현이 가능하다. 
$$
g(x) = input \\
f(x) = kernel \\
h(y) = output \\
h(y)\in[1,\lfloor(l-k)/d\rfloor+1] \\
h(y)=\sum^k_{x=1}f(x)\cdot g(y\cdot d-x+c)
$$
위에서 사용된 상수 $c$는 c=k-d+1 로 offset 상수이다. 또한 output $h_{j}(y)$는 모든 $i$에 대해서 input과 kernel을 convolution 연산하면서 얻어진다.
<br>

Deep-Learning 과정에서 6 layer의 깊이에서, max-pooling을 사용한다. 
$$
h(y)=\max^k_{x=1}g(y\cdot d-x+c)
$$
그리고 non-linearity를 위한 함수로 ReLU 함수를 사용하고, 128 크기의 minibatch로 Stochastic gradient descent를 사용했으며 모멘텀을 사용해 updata를 진행했다. 모멘텀 사용 시 모멘텀 상수는 0.9로 하였고, 학습률은 0.01에서 시작해서 3epoch 마다 절받으로 줄였다. 

## Character quantization

모델에서 인코딩된 글자들의 sequence를 input값으로 받았다. 여기서 인코딩은 m개의 알파벳에 대한 one-hot 인코딩방식이다. input은 m-Dimension이 된다. 만약 알파벳에 들어가지 않는 문자에 대해서는 0배거로 만든다. 특정 길이 까지만 input으로 받는다. 길이를 넘어가는 값에 대해서는 무시한다. 

## Model Design

2개의 ConvNet을 디자인 했으면, 하나는 많은 feature를 가지는 Conv고 하나는 적은 feature인 Conv다. 전체 모델은 9개 레이어로 구성되었으며, 6개를 Conv-layer 그리고 3개는 fully-connected layer로 구성되었다.
<br>

1. 하나의 문자당 70 Dimension(소문자)의 vector로 encoding이 진행된다. embedding 이 아니다.
    - 1014 개의 char를 입력 sequence로 받는다. 1014개로 문장의 글자가 끊기는건가? 
    - batch size, 70, 1014, 계행 문자도 같이 들어간다. 그냥 하나씩 들어간다. 짧은 문장은 패딩한다. 
    - 1014은 task에 성능을 찾아볼 수도 있다.
    - 리뷰의 길이의 히스토그램 중, 대표값을 사용한다.
    
2. input 70 by 1014로 구성한다. 의 단어 백터가 쌓인 매트릭스가 됩니다.
3. 1-d convolution 을 취함, 각 conv는 6개의 Conv + relu + Maxpool 의 형태입니다.


## Evaluation based Comparison

비교를 위한 모델은 전통적인 NLP 방식과 Deep Learning방식 두가지를 모두 사용해서 비교했다.
<br>

__Traditional Method__
- Bag of words and its TFIDF
- Bag of ngrams and its TFIDF
- Bag of means on word embedding
    - KNN
<br>

__Deep Learning Methods__
- Word based ConvNets
- LSTM
    - 마지막 스테이트를 합친다?

## Concolusion

- 글자 단위의 Convolutional Network도 문서 분류에서 높은 성능을 보인다.
- 작은 데이터셋에서는 전통적인 NLP방식이 DL방식보다 더 높은 성능을 보인다.
- ConvNet은 사용자가 만든 데이터에서 좋다.(오타를 잘 잡는다)
- Alphabet의 선택에 따라 성능이 많이 달라진다.
- Bag-of-means 모델은 안좋다.
- 모든 데이터셋에 있어 최적의 모델은 없다.( 많은 실험을 통해 데이터셋에 가장 적합한 모델을 찾아야 한다)


한글같은경우는 11000개의 charact인데, unicode index를 기준으로 Convolution을 하는게 어떨까?